In [66]:
import pandas as pd

## Peewee tutorial
Peewee is an object relational mapping framework. It connects a relational model (mysql) to objects (python).

### Make sure peewee is installed

In [67]:
!pip install peewee -q

### Install the mysql-python connector
This allows python to talk to the mysql server

In [68]:
!pip install pymysql -q

### Pwiz
pwiz allows you to convert the database structure to python objects.

Run the following command in the command line: `python -m pwiz -e mysql -H 165.22.199.122 -u remote -P masterclass`

Password: `EtrPCEc0jt`

To store it in a separate file: `python -m pwiz -e mysql -H 165.22.199.122 -u remote -P masterclass > models.py`

In [69]:
from peewee import *

database = MySQLDatabase('masterclass', **{'charset': 'utf8', 'sql_mode': 'PIPES_AS_CONCAT', 'use_unicode': True, 'host': '165.22.199.122', 'user': 'remote', 'password': 'EtrPCEc0jt'})

class UnknownField(object):
    def __init__(self, *_, **__): pass

class BaseModel(Model):
    class Meta:
        database = database

class Bananas(BaseModel):
    color = CharField(null=True)
    idbananas = AutoField()
    length = IntegerField(null=True)

    class Meta:
        table_name = 'bananas'

### Inserting one new record

In [70]:
Bananas.create(color='green', length=5)

<Bananas: 7>

### Inserting records in bulk
Let's first create a dataframe with the data

In [71]:
df = pd.DataFrame([{'color': 'yellow', 'length': 10}, {'color': 'red', 'length': 15}])

df

,color,length
0,yellow,10
1,red,15


#### Make sure you have the right orientation

In [72]:
data = df.to_dict(orient='records')
data

[{'color': 'yellow', 'length': 10}, {'color': 'red', 'length': 15}]

In [73]:
(Bananas
 .insert_many(data)
 .execute())

8

### Updating records

In [74]:
(Bananas
 .update(color='black')
 .where(Bananas.length < 12)
 .execute())

2

### Getting one specific record

In [75]:
from playhouse.shortcuts import model_to_dict

In [76]:
banana = Bananas.get(Bananas.color == 'red')

model_to_dict(banana)

{'idbananas': 9, 'color': 'red', 'length': 15}

### Getting multiple records

In [77]:
long_bananas = (Bananas
                .select()
                .where(Bananas.length > 5)
                .dicts())

for banana in long_bananas:
    print(banana)

{'idbananas': 8, 'color': 'black', 'length': 10}
{'idbananas': 9, 'color': 'red', 'length': 15}


### Deleting records

In [79]:
(Bananas
 .delete()
 .where(Bananas.color == 'black')
 .execute())

0

## Exercises

### Create a `models.py` file with all the models from the `masterclass` database

In [82]:
from models import Bananas

### Insert 5 bananas with different colors and lengths

### Get the average length of the bananas grouped by color